In [1]:
import cv2
import numpy as np
import os
from multiprocessing import Pool

In [2]:
#URL for downloading the videos
video_url = ''

# Step 1
The function getImages() is used to download a single .ts file or multiple .ts files in parallel and extracts the first frame as a .jpg image.

In [3]:
#This function accepts name of a video file, downloads it and extracts the first frame as a .jpg image
def extractImageFromVideo(file_name):
    try:
        print('downloading {}{}.ts...'.format(video_url,file_name))
        video = cv2.VideoCapture(video_url+file_name+'.ts')
        if(video.isOpened() == False):
            video.open(video_url+file_name+'.ts')  #Initialize the video if it did not open properly
        print('extracting image... ', end='')
        flag, frame = video.read()
        if(flag):        #Checking if the frame was read correctly or not
            cv2.imwrite(file_name+'.jpg',frame)
            print('wrote {}.jpg'.format(file_name))
            video.release()
            return True
        else:
            print('Error in reading the video file')
            video.release()
    except Exception as e:
        print('There is no video file named {}.ts'.format(file_name))
    return False

In [4]:
#This function downloads a single .ts file (concurrency=False) or multiple .ts files (concurrency=True) in parallel
def getImages(file_names, concurrency=False):
    if(concurrency):
        p = Pool()     
        return(p.map(extractImageFromVideo, file_names))  #file_names will be a list of video file names
    else:
        return(extractImageFromVideo(file_names))   #file_names will be the name of a single video file

# Step 2
The function detectCar() accepts file name of an image, runs the YOLO network and detects if there is a car or not at the specified parking spot. For doing this, I'm taking two reference points (similar to anchor boxes in YOLO) within the parking spot- one for the top left corner and other for the bottom right corner and then calculating L2 distance between the reference points and all the bounding boxes predicted by the YOLO network. Finally, if any one of these distances is less than some threshold value, I am predicting that there is a car at the parking spot. <b>For more details, please refer to the readme file. </b> </br>
<img src="./car_detection.png" width="500px" height="500px"/>

In [5]:
from math import sqrt
from darkflow.net.build import TFNet

In [6]:
#options to be specified for running the yolo network
option = {
    'model' : 'cfg/yolo.cfg',
    'load' : 'weights/yolo.weights',
    'threshold' : 0,
    'gpu' : 1.0
}

In [7]:
def L2(x1, y1, x2, y2):
    return (int(sqrt(pow(x1-x2,2)+pow(y1-y2,2))))

In [8]:
#Reference point for the top left corner and the bottom right corner are (214,219) and (241,231) respectively, with threshold value equal to 80 
def detectCar(file_name):
    network = TFNet(option)
    image = cv2.imread(file_name)
    print('running yolo on {}...'.format(file_name))
    result = network.return_predict(image)
    for i in range(len(result)):    #Looping through all the predictions
        info = result[i]
        if(info['label'] == 'car'):  
            topX = int(info['topleft']['x'])
            topY = int(info['topleft']['y'])
            bottomX = int(info['bottomright']['x'])
            bottomY = int(info['bottomright']['y'])
            l2_dist = L2(214,219,topX,topY) + L2(241,231,bottomX,bottomY)
            if(l2_dist<=80):
                print('car detected!')
                return
    print('no car detected!')

# Bonus
The function detectCarAtSpot() can be used to analyze any parking spot of your choice. It works similar to the detectCar() function except that it has reference points and threshold values for every parking spot. Over here, I have numbered parking spots (leaving ones at the extreme ends as they were often not being detected by the YOLO network) from 0-7 starting from the left-hand side of the image. Hence, the parking spot specified in this assignment gets the number 4. <b>For more details, please refer to the readme file. </b> </br>
<img src="./reference_points.jpg" width="500px" height="500px"/>

In [9]:
#Threshold value for all the parking spots
parking_thresholds = [60,60,60,60,80,72,70,45]
#Reference points (topX,topY,bottomX,bottomY) for all the parking spots
parking_spots = [(25,242,37,249),(56,236,75,245),(98,233,121,245),(145,225,173,236),(214,219,241,231),(637,209,680,231),(707,210,732,232),(752,214,782,240)]

In [10]:
#This function takes the file name of an image along with a parking spot number and detects if a car is present or not
def detectCarAtSpot(file_name, spot_no): 
    network = TFNet(option)
    image = cv2.imread(file_name)
    print('running yolo on {}...'.format(file_name))
    result = network.return_predict(image)
    for i in range(len(result)):    #Looping through all the predictions
        info = result[i]
        if(info['label'] == 'car'):
            topX = int(info['topleft']['x'])
            topY = int(info['topleft']['y'])
            bottomX = int(info['bottomright']['x'])
            bottomY = int(info['bottomright']['y'])
            l2_dist = L2(parking_spots[spot_no][0],parking_spots[spot_no][1],topX,topY) + L2(parking_spots[spot_no][2],parking_spots[spot_no][3],bottomX,bottomY)
            if(l2_dist<=parking_thresholds[spot_no]):
                print('car detected!')
                return
    print('no car detected!')

# Step 3
The function compareImages() accepts file name of two images and checks if both the images is of the same car or not. To do this, I’m first analyzing the number of cars that were parked within the given time range and if it is greater than one (meaning the car is not there anymore but can come again in future) then I am going for template matching because we know that those cars will be rotation and scale invariant and template matching works well when these conditions hold true. So, I’m cropping out the area containing the parking spot from the images, converting the cropped image into gray scale, normalizing and computing L2 distance between them (pixel wise). Finally, if this distance is less than some threshold value then I am predicting that both of these images are of the same car.

In [11]:
#This function will check if there was only one car present at the parking spot within the given time interval 
def sameCarHelper(from_time, to_time):
    cur_time = from_time
    while(cur_time<=to_time):
        check = getImages(str(cur_time))
        if(check):   #checking if a video exists with the given file_name or not
            detected = detectCar(str(cur_time)+'.jpg')
            if(detected==False): #if there is no car at the current time step means the car at time1 has left but it can come back later after some time
                return False
            cur_time = cur_time + 4
        else:
            cur_time = cur_time + 1  #increasing time by one if no video was found with the current time step
    return True

In [12]:
def normalizeImage(image):
    return ((image - image.min())/ (image.max() - image.min()))  #Normalizing Image

In [13]:
#This function takes the file name of two images and checks if they belong to the same car or not
def compareImages(file1_name, file2_name):
    time1 = int(file1_name[:file1_name.index('.jpg')])
    time2 = int(file2_name[:file2_name.index('.jpg')])
    check = False
    if(time1<=time2):
        check = sameCarHelper(time1, time2)
    else:
        check = sameCarHelper(time2, time1)
    print('comparing {} and {}... '.format(file1_name,file2_name), end='')
    if(check):
        print('same car!')
        return
    #If sameCarHelper returns false then we have to check if the car comes back again later
    image1 = cv2.imread(file1_name)
    image2 = cv2.imread(file2_name)
    cropped_image1 = image1[189:252,185:274]
    cropped_image2 = image2[189:252,185:274]
    gray_image1 = cv2.cvtColor(cropped_image1,cv2.COLOR_BGR2GRAY)
    gray_image2 = cv2.cvtColor(cropped_image2,cv2.COLOR_BGR2GRAY)
    normalized_image1 = normalizeImage(gray_image1)
    normalized_image2 = normalizeImage(gray_image2)
    l2_norm = np.sqrt(np.sum(np.power(normalized_image1-normalized_image2,2)))
    if(l2_norm<=10):
        print('same car!')
    else:
        print('not same car!')

# Step 4
The function analyzeCars() accepts a time range and outputs each car that was detected and how long it was parked for (approximately). To do this, I'm keeping the information of any car that was parked in the previous time step and if a car was detected or not at the current time step. With this knowledge, I'm predicting cars that were detected and for how long were they parked. <b>For more details, please refer to the readme file.</b>   

In [14]:
def analyzeCars(from_time, to_time):
    if(to_time-from_time < 0):
        print('Invalid time interval provided')
        return
    if(os.path.isdir('./output') == False):    #Checking if output folder existst or not
        os.mkdir('output')
    car_found = False  #this is used to check if a car was detected in the previous time steps
    parked_at = 0   #this is used to store the starting time of a parked car
    car = None  #this is used to store the frame containing a car when it will be found for the first time at the parking spot
    print('analyzing every car from {} to {}'.format(from_time,to_time))
    cur_time = from_time
    valid_endTime = from_time     #This will hold the correct end time for the last car (if to_time>max time in the database)
    while(cur_time<=to_time):
        check = getImages(str(cur_time))
        if(check):   #checking if a video exists with the given file_name or not
            detected = detectCar(str(cur_time)+'.jpg')
            if(detected==False and car_found): #if currently there is no car but car_found is true, means that the previously parked car has left
                car_found = False
                parking_time = cur_time-parked_at
                seconds = parking_time%60
                minutes = int((parking_time-seconds)/60)
                print('found car at {}. parked until {} ({} minutes and {} seconds).'.format(parked_at,cur_time,minutes,seconds))
                cv2.imwrite('output/{}-{}min{}sec.jpg'.format(parked_at,minutes,seconds),car)
                print('... wrote output/{}-{}min{}sec.jpg'.format(parked_at,minutes,seconds))
                parked_at = 0
            elif(detected and car_found == False): #if a car is detected and car_found is false, means that the car is detected for the first time
                car_found = True
                parked_at = cur_time
                car = cv2.imread(str(cur_time)+'.jpg')
            valid_endTime = cur_time
            cur_time = cur_time + 4
        else:
            cur_time = cur_time + 1  #increasing time by one if no video was found with the current time step
    if(parked_at != 0):  #checking if there was still a car parked at the end of the interval
        parking_time = valid_endTime-parked_at
        seconds = parking_time%60
        minutes = int((parking_time-seconds)/60)
        print('found car at {}. parked until {} ({} minutes and {} seconds).'.format(parked_at,valid_endTime,minutes,seconds))
        cv2.imwrite('output/{}-{}min{}sec.jpg'.format(parked_at,minutes,seconds),car)
        print('... wrote output/{}-{}min{}sec.jpg'.format(parked_at,minutes,seconds))
    print('no more cars found!')

# Bonus
The function detectColor() accepts file name of an image containing a car and predicts the color of the car. To do this, I'm first converting the image into the HSV color scheme, cropping out a small area containing the car, taking mean of the HSV values over the pixels from the cropped image and then using these average HSV values to predict different colors.

In [15]:
#This function takes the average HSV values of the cropped image and predicts the color of the car
def whichColor(hue, saturation, value):
    if(value==0 or (value<=100 and saturation<=100)):
        return 'Black'
    elif((value>=230 and saturation<140) or (value>=140 and saturation<40)):
        return 'White'
    elif(saturation<=100):
        return 'Grey'
    elif(hue>=0 and hue<=4):
        return 'Red'
    elif(hue>16 and hue<=34):
        return 'Yellow'
    elif((hue>34 and hue<=90) or (hue>204 and hue<=256) or (hue>288 and hue<=340)):
        return 'Green'
    elif((hue>90 and hue<=95) or (hue>341 and hue<=348)):
        return 'Greenish Blue'
    elif((hue>95 and hue<=120) or (hue>349 and hue<=360)):
        return 'Blue'
    elif((hue>120 and hue<=160)):
        return 'Purple'
    elif((hue>160 and hue<=184) or (hue>256 and hue<=262)):
        return 'Maroon'
    elif((hue>4 and hue<=16) or (hue>184 and hue<=204) or (hue>262 and hue<=288)):
        return 'Brown'
    else:
        return 'I cannot guess the color'

In [16]:
#This function detects color of the car
def detectColor(file_name):
    image = cv2.imread(file_name)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    avg_hue = np.mean(hsv_image[189:252,185:274,0])
    avg_saturation = np.mean(hsv_image[189:252,185:274,1])
    avg_value = np.mean(hsv_image[189:252,185:274,2])
    print(whichColor(avg_hue,avg_saturation,avg_value))